In [20]:
import numpy as np
import pickle
import random
import json
import os
import plotly.graph_objects as go
import kaleido

from tqdm.notebook import tqdm

# A Process class which emulates a process in a server. It has a pid and a length measured in numbers of instructions.
# Furthermore, it possesses the __repr__ method which is used to print the object.
# Moreover, it possesses a to_json method which is used to convert the object to a json string and a from_json to convert it back.

class Process:
    def __init__(self, pid, length):
        self.pid = pid
        self.length = length

    def __repr__(self):
        return f"Process(pid={self.pid}, length={self.length})"

    def to_json(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)
    
    @staticmethod
    def from_json(json_string):
        return json.loads(json_string)

In [21]:
# A Server class which emulates a server. It has a name, a number of cpus, the speed of its cpu measured in GHz and a workload list, which represents 
# the processes that are assigned to the server. The workload starts as empty. 
# Furthermore, it possesses the __repr__ method which is used to print the object.
# Moreover, it possesses a to_json method which is used to convert the object to a json string and a from_json to convert it back.

class Server:
    def __init__(self, name, cpus, cpu_speed):
        self.name = name
        self.cpus = cpus
        self.cpu_speed = cpu_speed

    def __repr__(self):
        return f"Server(name={self.name}, cpus={self.cpus}, cpu_speed={self.cpu_speed}"

    def to_json(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)
    
    @staticmethod
    def from_json(json_string):
        return json.loads(json_string)


In [22]:
# Function to find the magnitude of a vector.
def magnitude(vector):
    return np.sqrt(np.sum(np.square(vector)))

# Function to find the module of a vector.
def module(vector):
    return np.sum(np.square(vector))

# function to rescale vector between 0 and 1 and return it as a list
def rescale_vector(vector):
    return list((vector - np.min(vector)) / (np.max(vector) - np.min(vector)))

In [23]:

# A Solution class, wich represents a solution to the problem. is made of a list of tuples in the form <server, associated processes>
# The solution object possesses a fitness value, which is a vector of the fitness of each servers with respect to their associated processes. 
# The fitness is calculated as the sum of the length of the processes assigned to the server divided by the number of cpus of the server times their speed.
# It possesses the __repr__ method which is used to print the object.
# It possesses a to_json method which is used to convert the object to a json string and a from_json to convert it back.

class Solution:
    def __init__(self, solution):
        self.solution = solution
        self.fitness = rescale_vector(self.calculate_fitness())

    def __repr__(self):
        return f"Solution(solution={self.solution}, fitness={self.fitness})"

    def to_json(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)
    
    @staticmethod
    def from_json(json_string):
        return json.loads(json_string)

    def calculate_fitness(self):
        fitness = []
        for server, processes in self.solution:
            fitness.append(sum([process.length for process in processes]) * (server.cpus * server.cpu_speed))
        return fitness
        
    def __lt__(self, other):
        if magnitude(self.fitness) < magnitude(other.fitness):
            return 1
        elif magnitude(self.fitness) > magnitude(other.fitness):
            return -1
        else:
            return 0
        

In [24]:
# The instantiate_servers function randomly instantiate N servers and their parameters and returns them as a list. It does not instantiate the workloads. 
# The name of the server is a serial number that increases with i in range(n).

def instantiate_servers(n):
    servers = []
    for i in range(n):
        servers.append(Server(i, random.randint(1, 100), random.randint(1, 10)))
    return servers
    

In [25]:
# The instantiate_processes function randomly instantiate N processes and their parameters and returns them as a list. The PID is serial and depends on i.
# The length of the process is a random integer between 100 and 100000.

def instantiate_processes(n):
    processes = []
    for i in range(n):
        processes.append(Process(i, random.randint(10000, 10000000)))
    return processes

In [26]:
# A genetic algorithm comprised of all the necessary methods to perform the genetic algorithm.
# The algorithm is initialized with:
#   a list of servers,
#   a list of processes,
#   a population size,
#   a number of generations,
#   a mutation rate,
#   a crossover rate.
# At the very first iteration, the algorithm creates the starting population, which is a list of Solution objects. One single solution corresponds to one single chromosome.
# The algorithm then iterates through the generations and performs the following steps:
# 1. The chromosomes are selected for reproduction.
#       Remind to keep the dimensionality of the lists acceptable to not raise errors of dimensionality.
#       Make sure the list doesn't go out of range.
# 2. The chromosomes are reproduced.
# 3. The chromosomes are mutated.
#   The mutation overrides the current solution with a new one with the mutated parameters.
# 4. The chromosomes are sorted by fitness
# 5. The chromosomes are selected for the next generation.
#   The selection is made by taking the first n chromosomes, where n is the population size.
#   The population size is adjusted accordingly to the dimension of the population in each epoch.
# 6. The algorithm iterates through the generations until it reaches the maximum number of generations.
# 7. The algorithm returns the best chromosome of the last generation.
# The algorithm saves the fitness of the best chromosome in each generation in a json file and the first best solution produced and the best solution produced in the final iteration in json files, 
# all in the folder "./results/" with another subfolder called after the class of the genetic algorithm in question and contained in the self.name field.
# Make sure that the folders do exist and, if not, create them.
class GeneticAlgorithmBaseline:
    def __init__(self, servers, processes, population_size, generations, mutation_rate, crossover_rate):
        self.servers = servers
        self.processes = processes
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.name = "baseline"
        self.best_fitness = []

    def select(self, population):
        return random.choices(population, k=self.population_size)

    def reproduce(self, population):
        new_population = []
        for i in range(self.population_size):
            parent1 = random.choice(population)
            parent2 = random.choice(population)
            child = self.crossover(parent1, parent2)
            new_population.append(child)
        return new_population

    def crossover(self, parent1, parent2):
        if random.random() < self.crossover_rate:
            child = []
            for i in range(len(parent1.solution)):
                if random.random() < 0.5:
                    child.append(parent1.solution[i])
                else:
                    child.append(parent2.solution[i])
            return Solution(child)
        else:
            return parent1

    def mutate(self, population):
        for i in range(len(population)):
            if random.random() < self.mutation_rate:
                population[i] = self.random_solution()

    def random_solution(self):
        solution = []
        for server in self.servers:
            processes = []
            for process in self.processes:
                if random.random() < 0.5:
                    processes.append(process)
            solution.append((server, processes))
        return Solution(solution)

    # The sort function operates by taking into account the vectorial magnitude of the fitness of the chromosome
    # It must pay attention not to do a square root of a negative number, which would raise an error.
    # The magnitude of the 
    def sort(self, population):
        return sorted(population, key=lambda chromosome: magnitude(chromosome.fitness), reverse=True)

        
    def select_next_generation(self, population):
        return population[:self.population_size]

    def run(self, iteration):
        population = [self.random_solution() for i in range(self.population_size)]
        for i in tqdm(range(self.generations)):
            population = self.select(population)
            population = self.reproduce(population)
            self.mutate(population)
            population = self.sort(population)
            population = self.select_next_generation(population)
            self.best_fitness.append(population[0].fitness)
        with open(f"./results/{self.name}/best_fitness_{iteration}.json", "w") as f:
            f.write(json.dumps(self.best_fitness, indent=4))
        with open(f"./results/{self.name}/starting_best_solution_{iteration}.json", "w") as f:
            f.write(population[0].to_json())
        with open(f"./results/{self.name}/final_best_solution_{iteration}.json", "w") as f:
            f.write(population[0].to_json())
        return population[0]


In [27]:
# An improvement of the above genetic algorithm called GeneticAlgorithmElitism which implement the elitism strategy.
# The folders and paths change accordingly.

class GeneticAlgorithmElitism(GeneticAlgorithmBaseline):
    def __init__(self, servers, processes, population_size, generations, mutation_rate, crossover_rate):
        super().__init__(servers, processes, population_size, generations, mutation_rate, crossover_rate)
        self.name = "elitism"

    def select_next_generation(self, population):
        return population[:int(self.population_size / 2)] + population[-int(self.population_size / 2):]
    
    def run(self, iteration):
        super().run(iteration)

In [28]:
# An improvement of the GeneticAlgorithmElitism genetic algorithm called GeneticAlgorithmTournament which implements the tournament strategy.

class GeneticAlgorithmTournament(GeneticAlgorithmElitism):
    def __init__(self, servers, processes, population_size, generations, mutation_rate, crossover_rate):
        super().__init__(servers, processes, population_size, generations, mutation_rate, crossover_rate)
        self.name = "tournament"

    def select(self, population):
        return [max(random.choices(population, k=2)) for i in range(self.population_size)]
    
    def run(self, iteration):
        super().run(iteration)

In [29]:
# An improvement of the GeneticAlgorithmTournament genetic algorithm called GeneticAlgorithmDynamic, 
# which implements the dynamic adaptation of the parameters of the genetic algorithm.

class GeneticAlgorithmDynamic(GeneticAlgorithmTournament):
    def __init__(self, servers, processes, population_size, generations, mutation_rate, crossover_rate):
        super().__init__(servers, processes, population_size, generations, mutation_rate, crossover_rate)
        self.name = "dynamic"
        self.mutation_rate = 0.1
        self.crossover_rate = 0.5
        self.generations = 50

    def run(self, iteration):
        population = [self.random_solution() for i in range(self.population_size)]
        for i in tqdm(range(self.generations)):
            population = self.select(population)
            population = self.reproduce(population)
            self.mutate(population)
            population = self.sort(population)
            population = self.select_next_generation(population)
            self.best_fitness.append(population[0].fitness)
            self.mutation_rate = 1 / (1 + np.exp(-population[0].fitness[0]))
            self.crossover_rate = 1 / (1 + np.exp(-population[0].fitness[1]))
            self.generations = int(50 / (1 + np.exp(-population[0].fitness[2])))
        with open(f"./results/{self.name}/best_fitness_{iteration}.json", "w") as f:
            f.write(json.dumps(self.best_fitness, indent=4))
        with open(f"./results/{self.name}/starting_best_solution_{iteration}.json", "w") as f:
            f.write(population[0].to_json())
        with open(f"./results/{self.name}/final_best_solution_{iteration}.json", "w") as f:
            f.write(population[0].to_json())
        return population[0]

In [30]:
# The Execute function randomly instantiates 10 servers, 1000 processes (these two by using the related functions),
# a population size of 100, 25 generations, a mutation rate of 0.01 and a crossover rate of 0.9.
# The servers are saved as a pickle file in the path "./servers/" and the processes as pickles in the path "./processes/".
# Do all of this in a for loop with a range of 10, every time saving the servers and processes in the same folder but with a different name,
# that increases with the loop index.
# In another for loop with a range of 10, load the servers and processes from the folders created in the previous loop.
# At each iteration load a pickle for the servers, a pickle for the processes and run the genetic algorithm with the servers and processes.
# If the parameter "initialize" is True, then it saves the servers and processes pickles and then runs the genetic algorithm.
# If the parameter "initialize" is False, then it jsut runs the genetic algorithm.
# Accordingly to the "ga" parameter, it runs a different genetic algorithm.
# Before running an algorithm, create all the folder that the algorithm will need.

def execute(ga, num_epochs, pop_size, initialize):
    for i in tqdm(range(10)):
        servers = instantiate_servers(10)
        processes = instantiate_processes(10000)
        if initialize:
            with open(f"./servers/{i}.pickle", "wb") as f:
                pickle.dump(servers, f)
            with open(f"./processes/{i}.pickle", "wb") as f:
                pickle.dump(processes, f)
    for j in tqdm(range(10)):
        with open(f"./servers/{i}.pickle", "rb") as f:
            servers = pickle.load(f)
        with open(f"./processes/{i}.pickle", "rb") as f:
            processes = pickle.load(f)
        if ga == "baseline":
            algorithm = GeneticAlgorithmBaseline(servers, processes, pop_size, num_epochs, 0.01, 0.9)
        elif ga == "elitism":
            algorithm = GeneticAlgorithmElitism(servers, processes, pop_size, num_epochs, 0.01, 0.9)
        elif ga == "tournament":
            algorithm = GeneticAlgorithmTournament(servers, processes, pop_size, num_epochs, 0.01, 0.9)
        elif ga == "dynamic":
            algorithm = GeneticAlgorithmDynamic(servers, processes, pop_size, num_epochs, 0.01, 0.9)
        """elif ga == "parallel":
            algorithm = GeneticAlgorithmParallel(servers, processes, 100, 25, 0.01, 0.9)"""
        #os.makedirs(f"./results/{algorithm.name}/", exist_ok=True)
        algorithm.run(j)

In [32]:
# if __name__ == "__main__": run the execute function. Then plot directly the all the graphs.

if __name__ == "__main__":
    execute("baseline", 500, 200, False)
    #plot_fitnesses("./results/baseline/best_fitnesses.json")
    #plot_clusters("./results/baseline/starting_best_solution.json", "./results/baseline/final_best_solution.json")

    execute("elitism", 500, 200, False)
    #plot_fitnesses("./results/elitism/fitnesses.json")
    #plot_clusters("./results/elitism/start.json", "./results/elitism/end.json")

    execute("tournament", 500, 200, False)
    #plot_fitnesses("./results/tournament/fitnesses.json")
    #plot_clusters("./results/tournament/start.json", "./results/tournament/end.json")

    #execute("dynamic", 10, False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [16]:
if __name__ == "__main__":
    execute("dynamic", 500, 200, False)
    #plot_fitnesses("./results/dynamic/fitnesses.json")
    #plot_clusters("./results/dynamic/start.json", "./results/dynamic/end.json")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# A function that takes the path to a json file that contains a list of lists. Then does the transpose of the matrix and returns it as a list of lists.
def transpose_matrix(path):
    matrix = []
    with open(path, 'r') as file:
        matrix = json.load(file)
    return np.asarray(matrix).T

In [ ]:
def plot_results(path, n):
   fig = go.Figure()
   for i in range(n):
      a = transpose_matrix(path)[i]
      val = a*(10**abs(np.mean(extract_exponential_part(a))))
      fig.add_trace(go.Scatter(y=val, mode='lines', name=f'Run {i}'))
   fig.update_layout(title='Best fitness of the baseline algorithm',
                     xaxis_title='Epoch',
                        yaxis_title='Fitness')
   fig.show()

plot_results('./results/baseline/best_fitness_0.json', 10)

In [18]:
# Given the path of the results, this function plots the fitnesses of the best chromosome over the generations using the library plotly.
# The file is a json file containing a list of lists of fitnesses. For each sublist, calculate the magnitude of the sublist. 
# Then, against the x-axis, stands the number of the generation and against the y-axis stands the magnitude of the fitness sublist.
# To better visualize the results, the y-axis is in logarithmic scale.
# The image is saved to the specified path given as a parameter.

def plot_fitnesses(path, destination, slice=10):
    with open(path, "r") as f:
        data = json.loads(f.read())
    data = [magnitude(x) for x in data][:slice]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=[i for i in range(len(data))], y=data, mode="lines"))
    fig.update_layout(
        title="Fitnesses of the best chromosome over the generations",
        xaxis_title="Generation",
        yaxis_title="Fitness",
        yaxis_type="log"
    )
    # Use of the to_image function of the plotly library to save the image to the specified path.
    fig.write_image(destination, engine="kaleido")
    fig.show()

In [16]:
# This function is like the plot_fitness function, but given a folder and a filename, 
# it crawls all the subfolders to find all the paths of the files with the specified filename.
# Then, it plots in the same graph all the fitnesses over epochs of the files with the same name. Exclude the dynamic folder.

def plot_fitnesses_folder(folder, filename):
    paths = []
    for root, dirs, files in os.walk(folder):
        if "dynamic" in root:
            continue
        for file in files:
            if file == filename:
                paths.append(os.path.join(root, file))
    fig = go.Figure()
    for path in paths:
        with open(path, "r") as f:
            fitnesses = json.loads(f.read())
        fitnesses = [magnitude(fitness) for fitness in fitnesses]
        fig.add_trace(go.Scatter(x=list(range(len(fitnesses))), y=fitnesses, name=path))
    fig.update_layout(
        title="Fitnesses of the best chromosome over the generations",
        xaxis_title="Generation",
        yaxis_title="Fitness",
        yaxis_type="log"
    )
    fig.show()

In [17]:
# Given the path to the starting population and the path to the ending population, 
# this function plots two graphs of clustering of the processes in the starting and ending population.

def plot_clusters(start_path, end_path):
    with open(start_path, "r") as f:
        start = json.load(f)
    with open(end_path, "r") as f:
        end = json.load(f)
    fig = go.Figure()
    for chromosome in start:
        x = []
        y = []
        for process in chromosome:
            x.append(process["pid"])
            y.append(process["length"])
        fig.add_trace(go.Scatter(x=x, y=y, mode="markers", name="Start"))
    fig.update_layout(title="Start", xaxis_title="Process ID", yaxis_title="Process Length")
    fig.show()
    fig = go.Figure()
    for chromosome in end:
        x = []
        y = []
        for process in chromosome:
            x.append(process["pid"])
            y.append(process["length"])
        fig.add_trace(go.Scatter(x=x, y=y, mode="markers", name="End"))
    fig.update_layout(title="End", xaxis_title="Process ID", yaxis_title="Process Length")
    fig.show()

In [18]:
# A function to average, give a folder, all the values contained in the best_finesses_{}.json files and plot them.

def plot_all_fitnesses_folder(folder):
    paths = [f"{folder}/best_fitness_{i}.json" for i in range(10)]
    fig = go.Figure()
    for path in paths:
        with open(path, "r") as f:
            fitnesses = json.loads(f.read())
        fitnesses = [magnitude(fitness) for fitness in fitnesses]
        fig.add_trace(go.Scatter(x=list(range(len(fitnesses))), y=fitnesses, name=path))
    fig.update_layout(
        title="Fitnesses of the best chromosome over the generations",
        xaxis_title="Generation",
        yaxis_title="Fitness",
        yaxis_type="log"
    )
    fig.show()


In [19]:
# A function to read, given a folder, all the values contained in the best_finesses_{}.json files. Then performs the vectorial average over the axis 0 and plots them.

def plot_average_fitnesses_folder(folder):
    paths = [f"{folder}/best_fitness_{i}.json" for i in range(10)]
    all_fitnesses = []
    for path in paths:
        with open(path, "r") as f:
            fitnesses = json.loads(f.read())
        fitnesses = [magnitude(fitness) for fitness in fitnesses]
        all_fitnesses.append(fitnesses)
    all_fitnesses = np.array(all_fitnesses)
    average_fitnesses = np.average(all_fitnesses, axis=0)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(len(average_fitnesses))), y=average_fitnesses, name=path))
    fig.update_layout(
        title="Fitnesses of the best chromosome over the generations",
        xaxis_title="Generation",
        yaxis_title="Fitness",
        yaxis_type="log"
    )
    fig.show()

In [19]:
for i in tqdm(range(10)):
    plot_fitnesses("./results/baseline/best_fitness_{}.json".format(i), "./results/baseline/graphs/pop_size_200/500_epochs/fitness_{}.png".format(i), 500)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
plot_fitnesses_folder("./results/baseline/", "best_fitness_0.json")

In [ ]:
plot_fitnesses("./results/baseline/best_fitness_5.json")

In [ ]:
plot_all_fitnesses_folder("./results/baseline")

In [ ]:
plot_average_fitnesses_folder("./results/baseline")

  0%|          | 0/10 [00:00<?, ?it/s]

In [33]:
for i in tqdm(range(10)):
    plot_fitnesses("./results/elitism/best_fitness_{}.json".format(i), "./results/elitism/graphs/500_epochs/fitness_{}.png".format(i), 500)

  0%|          | 0/10 [00:00<?, ?it/s]

In [34]:
for i in tqdm(range(10)):
    plot_fitnesses("./results/elitism/best_fitness_{}.json".format(i), "./results/elitism/graphs/500_epochs/fitness_{}.png".format(i), 500)

  0%|          | 0/10 [00:00<?, ?it/s]